<a href="https://colab.research.google.com/github/cnhzgb/MachineL/blob/main/VGG9_32*32_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipdb
!pip install einops
import ipdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from einops import rearrange
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 591.9 kB/s eta 0:00:00


In [2]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) # transforms.RandomHorizontalFlip(), transforms.RandomGrayscale()
dataset = datasets.CIFAR10(root="dataset/", transform=trans, download=True, train=True) # 5W张图片, 10种分类
loader = DataLoader(dataset, batch_size=100, shuffle=True)
batch_num,(image, label) = next(enumerate(loader))
print(len(dataset.classes), len(dataset), image.shape, label.shape) # 10; 5W; 100,3,32,32; 100

trans_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
dataset_test = datasets.CIFAR10(root="dataset/", transform=trans_test, download=True, train=False) # 1W张图片
loader_test = DataLoader(dataset_test, batch_size=100, shuffle=True)
criterion_test = nn.CrossEntropyLoss()

100%|██████████| 170498071/170498071 [00:05<00:00, 29170380.31it/s]


Extracting dataset/cifar-10-python.tar.gz to dataset/
10 50000 torch.Size([100, 3, 32, 32]) torch.Size([100])
Files already downloaded and verified


In [3]:
# https://zhuanlan.zhihu.com/p/87555358
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.conv11 = nn.Conv2d(3,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv21 = nn.Conv2d(64,128,3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv31 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv32 = nn.Conv2d(256, 256, 3,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv41 = nn.Conv2d(256, 512, 3,padding=1)
        self.conv42 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv51 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv52 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(512*2*2,1024)
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024,10)


    def forward(self,x):
        x = F.relu(self.conv11(x)) # => ,64,32,32
        x = self.pool1(x) # => ,64,16,16

        x = F.relu(self.conv21(x)) # => ,128,16,16
        x = self.pool2(x) # => ,128,8,8

        x = F.relu(self.conv31(x)) # => ,256,8,8
        x = F.relu(self.conv32(x))
        x = self.pool3(x) # => ,256,4,4

        x = F.relu(self.conv41(x)) # => ,512,4,4
        x = F.relu(self.conv42(x))
        x = self.pool4(x) # => ,512,2,2

        #x = F.relu(self.conv51(x)) # => ,512,2,2
        #x = F.relu(self.conv52(x))
        # x = self.pool5(x) # => ,512,1,1

        x = x.view(-1,512*2*2)
        x = F.relu(self.fc1(x)) # => ,1024
        x = F.relu(self.fc2(x)) # => ,1024
        x = self.fc3(x) # => ,10

        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
         MaxPool2d-2           [-1, 64, 16, 16]               0
            Conv2d-3          [-1, 128, 16, 16]          73,856
         MaxPool2d-4            [-1, 128, 8, 8]               0
            Conv2d-5            [-1, 256, 8, 8]         295,168
            Conv2d-6            [-1, 256, 8, 8]         590,080
         MaxPool2d-7            [-1, 256, 4, 4]               0
            Conv2d-8            [-1, 512, 4, 4]       1,180,160
            Conv2d-9            [-1, 512, 4, 4]       2,359,808
        MaxPool2d-10            [-1, 512, 2, 2]               0
           Linear-11                 [-1, 1024]       2,098,176
           Linear-12                 [-1, 1024]       1,049,600
           Linear-13                   [-1, 10]          10,250
Total params: 7,658,890
Trainable param

In [4]:
for epoch in range(1,10):
  total_loss = []
  errorTotal = 0
  for batch_idx, (img, label) in enumerate(loader):
    img = img.to(device)
    label = label.to(device)
    outputs = model(img)
    loss = criterion(outputs, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss.append(loss.item())
    maxV,maxIdx = outputs.max(dim=1)
    errorNum = torch.sum(torch.ne(maxIdx, label)).item()
    errorTotal += errorNum

    if(batch_idx % 100 == 99):
      print("epoch:{} batch:{} loss:{:.2f} mean:{:.2f} error:{}/100 errorTotal:{}/{} {:.2f}%".format(
          epoch, batch_idx, loss, np.mean(total_loss), errorNum, errorTotal, (batch_idx+1)*100, errorTotal/(batch_idx+1)))

epoch:1 batch:99 loss:1.94 mean:2.10 error:80/100 errorTotal:7982/10000 79.82%
epoch:1 batch:199 loss:1.62 mean:1.95 error:60/100 errorTotal:14834/20000 74.17%
epoch:1 batch:299 loss:1.74 mean:1.87 error:63/100 errorTotal:21274/30000 70.91%
epoch:1 batch:399 loss:1.57 mean:1.80 error:56/100 errorTotal:27317/40000 68.29%
epoch:1 batch:499 loss:1.50 mean:1.75 error:61/100 errorTotal:33064/50000 66.13%
epoch:2 batch:99 loss:1.44 mean:1.49 error:56/100 errorTotal:5564/10000 55.64%
epoch:2 batch:199 loss:1.35 mean:1.46 error:51/100 errorTotal:10878/20000 54.39%
epoch:2 batch:299 loss:1.23 mean:1.44 error:45/100 errorTotal:16026/30000 53.42%
epoch:2 batch:399 loss:1.37 mean:1.42 error:52/100 errorTotal:21014/40000 52.53%
epoch:2 batch:499 loss:1.37 mean:1.39 error:51/100 errorTotal:25774/50000 51.55%
epoch:3 batch:99 loss:1.24 mean:1.26 error:47/100 errorTotal:4609/10000 46.09%
epoch:3 batch:199 loss:1.46 mean:1.24 error:50/100 errorTotal:9025/20000 45.12%
epoch:3 batch:299 loss:1.27 mean:1.